In [96]:
import pandas as pd

## importing data

In [97]:
df_mvps = pd.read_csv("mvps.csv")
df_mvps.head()

,Unnamed: 0,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,...,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,year
0,0,1,Michael Jordan,27,CHI,77,891,960,0.928,82,...,6.0,5.5,2.7,1.0,0.539,0.312,0.851,20.3,0.321,1991
1,1,2,Magic Johnson,31,LAL,10,497,960,0.518,79,...,7.0,12.5,1.3,0.2,0.477,0.320,0.906,15.4,0.251,1991
2,2,3,David Robinson,25,SAS,6,476,960,0.496,82,...,13.0,2.5,1.5,3.9,0.552,0.143,0.762,17.0,0.264,1991
3,3,4,Charles Barkley,27,PHI,2,222,960,0.231,67,...,10.1,4.2,1.6,0.5,0.570,0.284,0.722,13.4,0.258,1991
4,4,5,Karl Malone,27,UTA,0,142,960,0.148,82,...,11.8,3.3,1.1,1.0,0.527,0.286,0.770,15.5,0.225,1991


In [98]:
df_player = pd.read_csv("playerStats.csv")
df_player = df_player[df_player["Player"] != "Player"]
#df_player = df_player.dropna(subset=["Team"])
df_player.head()

,Unnamed: 0,Rk,Player,Age,Team,Pos,G,GS,MP,FG,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards,year
0,0,1,Michael Jordan,27,CHI,SG,82,82,37.0,12.1,...,4.6,6.0,5.5,2.7,1.0,2.5,2.8,31.5,"MVP-1,DPOY-7,AS,NBA1",1991
1,1,2,Karl Malone,27,UTA,PF,82,82,40.3,10.3,...,8.9,11.8,3.3,1.1,1.0,3.0,3.3,29.0,"MVP-5,AS,NBA1",1991
2,2,3,Bernard King,34,WSB,SF,64,64,37.5,11.1,...,3.2,5.0,4.6,0.9,0.3,4.0,2.9,28.4,"MVP-16,AS,NBA3",1991
3,3,4,Charles Barkley,27,PHI,SF,67,67,37.3,9.9,...,6.3,10.1,4.2,1.6,0.5,3.1,2.6,27.6,"MVP-4,AS,NBA1",1991
4,4,5,Patrick Ewing,28,NYK,C,81,81,38.3,10.4,...,8.8,11.2,3.0,1.0,3.2,3.6,3.5,26.6,"MVP-11,DPOY-7,AS,NBA2",1991


In [99]:
df_player_advanced = pd.read_csv("playerStatsAdvanced.csv")
df_player_advanced = df_player_advanced[df_player_advanced["Player"] != "Player"]
df_player_advanced.head()

,Unnamed: 0,Rk,Player,Age,Team,Pos,G,GS,MP,PER,...,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Awards,year
0,0,1,Chris Mullin,27,GSW,SF,82,82,3315,21.4,...,9.7,2.4,12.2,.176,4.5,0.1,4.7,5.6,"AS,NBA2",1991
1,1,2,Karl Malone,27,UTA,PF,82,82,3302,24.8,...,9.9,5.6,15.5,.225,4.8,0.6,5.4,6.2,"MVP-5,AS,NBA1",1991
2,2,3,Tim Hardaway,24,GSW,PG,82,82,3215,20.9,...,7.4,2.5,9.9,.148,4.5,-0.1,4.4,5.2,"MVP-19,AS",1991
3,3,4,Tyrone Corbin,28,MIN,SF,82,82,3196,16.6,...,2.6,2.9,5.4,.081,0.5,0.4,0.9,2.3,NaN,1991
4,4,5,Pooh Richardson,24,MIN,PG,82,82,3154,18.1,...,4.5,1.3,5.9,.089,3.1,-1.1,2.0,3.2,NaN,1991


# preprocessing standings:

In [100]:
df_standings = pd.read_csv('standings.csv')
df_standings.head()
df_standings['Team'] = df_standings['Team'].str.rstrip('*')
print(len(df_standings["Team"].unique()))
len(df_player["Team"].unique())

37


43

# Preprocessing: PlayerStats

In [101]:
# Combining entries of players who played on multiple teams in the same year
new_df_player = df_player.copy()

# Identify aggregate rows
new_df_player["is_aggregate"] = new_df_player["Team"].str.match(r"^[2-8]TM$")

def filter_and_replace_team(group):
    if group["is_aggregate"].any():
        # Find the aggregate row
        aggregate_row = group.loc[group["is_aggregate"]]
        
        # Find the Team value from the row with the highest index
        highest_index_team = group.loc[group.index.max(), "Team"]
        
        # Replace the Team value in the aggregate row with the highest index Team value
        aggregate_row = aggregate_row.copy()
        aggregate_row["Team"] = highest_index_team
        
        return aggregate_row
    else:
        # Return the whole group if no aggregate row exists
        return group

# Group by Player and Year, apply the filtering and replacing logic
new_df_player = (
    new_df_player.groupby(["Player", "year"], group_keys=False)
    .apply(filter_and_replace_team)
)

# Drop the is_aggregate column as it's no longer needed
new_df_player = new_df_player.drop(columns=["is_aggregate"])
new_df_player_adv = new_df_player_adv.dropna(subset=["Team"])

/var/folders/g9/3tj7x28j4hq3r0r9fqgmfcy80000gn/T/ipykernel_1697/2595381954.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(filter_and_replace_team)


In [102]:
new_df_player["Team"].unique()

array(['LAL', 'PHO', 'DAL', 'MIA', 'CLE', 'WSB', 'MIL', 'CHI', 'GSW',
       'IND', 'WAS', 'MIN', 'BOS', 'ATL', 'HOU', 'DEN', 'ORL', 'NOH',
       'TOR', 'SAC', 'CHO', 'PHI', 'POR', 'DET', 'OKC', 'UTA', 'VAN',
       'SEA', 'NJN', 'NOK', 'LAC', 'CHA', 'MEM', 'NYK', 'NOP', 'BRK',
       'SAS', 'CHH', nan], dtype=object)

In [74]:
#Dropping unnecessary columns for ML model
#new_df_player = new_df_player.drop(columns=["Team", 'Unnamed: 0', 'Rk', 'Team', 'Awards', 'Pos', 'Age'])

#Converting string values to numerical values for appropriate columsn
cols_to_convert = [col for col in new_df_player.columns if col not in ["Player"]]
new_df_player[cols_to_convert] = new_df_player[cols_to_convert].apply(pd.to_numeric, errors='coerce')

#Filling blank or null values with 0
new_df_player.fillna(0, inplace=True)

# Preprocessing: PlayerStatsAdvances

In [103]:
# Combining entries of players who played on multiple teams in the same year
new_df_player_adv = df_player_advanced.copy()
new_df_player_adv["is_aggregate"] = new_df_player_adv["Team"].str.match(r"^[2-8]TM$")
def filter_entries(group):
    grouping_columns = ["Player", "year"]
    data_columns = group.drop(columns=grouping_columns)
    if group["is_aggregate"].any():
        # Find the aggregate row
        aggregate_row = group.loc[group["is_aggregate"]]
        
        # Find the Team value from the row with the highest index
        highest_index_team = group.loc[group.index.max(), "Team"]
        
        # Replace the Team value in the aggregate row with the highest index Team value
        aggregate_row = aggregate_row.copy()
        aggregate_row["Team"] = highest_index_team
        
        return aggregate_row
    else:
        return group
new_df_player_adv = (
    new_df_player_adv.groupby(["Player", "year"], group_keys=False)
    .apply(filter_entries)
)
new_df_player_adv = new_df_player_adv.drop(columns=["is_aggregate"])

/var/folders/g9/3tj7x28j4hq3r0r9fqgmfcy80000gn/T/ipykernel_1697/204433328.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(filter_entries)


In [106]:
new_df_player["Team"].unique()
len(new_df_player_adv)
new_df_player_adv = new_df_player_adv.dropna(subset=["Team"])

15818

In [ ]:
#Dropping unnecessary columns for ML model
new_df_player_adv = new_df_player_adv.drop(columns=["Team", 'Unnamed: 0', 'Rk', 'Team', 'Awards', 'Pos', 'G', 'GS', 'MP', 'Age'])

#Converting string values to numerical values for appropriate columsn
cols_to_convert = [col for col in new_df_player_adv.columns if col not in ["Player"]]
new_df_player_adv[cols_to_convert] = new_df_player_adv[cols_to_convert].apply(pd.to_numeric, errors='coerce')

#Filling blank or null values with 0
new_df_player_adv.fillna(0, inplace=True)


In [76]:
len(new_df_player_adv)

15852

# preprocessing: MVP Data

In [78]:
new_df_player["Team"]

182     0.0
550     0.0
1035    0.0
1465    0.0
2021    0.0
       ... 
5903    0.0
6405    0.0
7090    0.0
7611    0.0
8224    0.0
Name: Team, Length: 15852, dtype: float64

In [150]:
df_mvps.columns
new_df_mvp = df_mvps.drop(columns=['Unnamed: 0','Age', 'Rank', 'Tm', 'First', 'Pts Won', 'Pts Max', 'G',
    'MP','PTS', 'TRB', 'AST', 'STL', 'BLK', 'FG%', '3P%', 'FT%', 'WS', 'WS/48'])

#Converting string values to numerical values for appropriate columsn
cols_to_convert = [col for col in new_df_mvp.columns if col not in ["Player"]]
new_df_mvp[cols_to_convert] = new_df_mvp[cols_to_convert].apply(pd.to_numeric, errors='coerce')


Index(['Player', 'Share', 'year'], dtype='object')

In [151]:
#merging advanced and regular stats
print(new_df_player.columns)
print(new_df_player_adv.columns)
df = pd.merge(new_df_player, new_df_player_adv, on=['Player', 'year'])
#filtering out players who started less than or equal to 40 games
df = df[df['GS'] > 40]

#normalizing data
columns_to_normalize = [col for col in df.columns if col not in ["Player", "year"]]

def normalize_year(group):
    for col in columns_to_normalize:
        group[col] = (group[col] - group[col].mean()) / group[col].std()
    return group

df = df.groupby('year', group_keys=False).apply(normalize_year)

Index(['Player', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P',
       '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS', 'year'],
      dtype='object')
Index(['Player', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%',
       'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM',
       'DBPM', 'BPM', 'VORP', 'year'],
      dtype='object')


/var/folders/g9/3tj7x28j4hq3r0r9fqgmfcy80000gn/T/ipykernel_96191/1394788153.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('year', group_keys=False).apply(normalize_year)


In [1]:
#merging player stats with MVP share
print(df.columns)
df = pd.merge(df, new_df_mvp, on=['Player', 'year'], how='left')
print(df.columns)
# Fill NaN values in columns from df2 with 0
df = df.fillna(0)

NameError: name 'df' is not defined

# PRE PROCESSING standings data

In [3]:
df_standings = pd.read_csv('standings.csv')
df_standings.head()

,Unnamed: 0,Team,W,L,W/L%,GB,PS/G,PA/G,SRS,year
0,0,Utah Jazz*,55,27,0.671,—,96.5,92.0,4.52,1991
1,1,San Antonio Spurs*,53,29,0.646,2.0,96.2,90.2,5.92,1991
2,2,Minnesota Timberwolves*,50,32,0.610,5.0,98.5,96.0,2.67,1991
3,3,Dallas Mavericks,40,42,0.488,15.0,101.4,102.0,-0.29,1991
4,4,Denver Nuggets,35,47,0.427,20.0,99.0,101.1,-1.76,1991


In [149]:
df.to_csv('datav1.csv')